# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770490374856                   -0.53    9.0    181ms
  2   -2.771691892925       -2.92       -1.32    1.0    104ms
  3   -2.771714322767       -4.65       -2.50    1.0   97.1ms
  4   -2.771714641189       -6.50       -3.25    1.0    515ms
  5   -2.771714712298       -7.15       -3.78    2.0    119ms
  6   -2.771714715127       -8.55       -4.79    1.0    133ms
  7   -2.771714715249       -9.91       -5.54    2.0    152ms
  8   -2.771714715249      -12.68       -5.86    1.0    100ms
  9   -2.771714715250      -12.58       -6.85    1.0    104ms
 10   -2.771714715250      -13.80       -7.19    2.0    116ms
 11   -2.771714715250   +  -13.92       -7.49    1.0    106ms
 12   -2.771714715250   +  -14.27       -7.68    1.0    109ms
 13   -2.771714715250      -13.80       -7.83    1.0    106ms
 14   -2.771714715250   +  -14.40       -8.51    1.0    121ms


-0.00013457346727059114

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770424241961                   -0.53    9.0    220ms
  2   -2.771770321028       -2.87       -1.31    1.0    154ms
  3   -2.771801510218       -4.51       -2.61    1.0    141ms
  4   -2.771802048724       -6.27       -4.16    1.0    461ms
  5   -2.771802074441       -7.59       -4.78    2.0    111ms
  6   -2.771802074474      -10.49       -5.73    1.0    110ms
  7   -2.771802074476      -11.65       -5.97    2.0    125ms
  8   -2.771802074476      -12.99       -6.53    1.0   99.1ms
  9   -2.771802074476      -13.77       -7.30    1.0    101ms
 10   -2.771802074476   +  -13.99       -7.98    2.0    117ms
 11   -2.771802074476      -14.10       -8.32    1.0    104ms


0.017612221492651303

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457346727059114
Displaced dipole:  0.017612221492651303
Polarizability :   1.7746794959921894


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.29e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 5.05e-09
[ Info: GMRES linsolve in iteration 2; step 1: normres = 8.20e-10
[ Info: GMRES linsolve in iteration 2; step 2: normres = 6.51e-11
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ * norm of residual = 1.35e-12
└ * number of operations = 12
Non-interacting polarizability: 1.925712538162061
Interacting polarizability:     1.7736548689176523


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.